# Setup

In [1]:
%cd /kaggle/working/
%rm -rf mlproduction
!git clone https://github.com/daoanhkhoa123/mlproduction.git
%cd /kaggle/working/mlproduction
!ls

/kaggle/working
Cloning into 'mlproduction'...
remote: Enumerating objects: 417, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 417 (delta 55), reused 71 (delta 30), pack-reused 317 (from 1)
Receiving objects: 100% (417/417), 3.60 MiB | 11.14 MiB/s, done.
Resolving deltas: 100% (243/243), done.
/kaggle/working/mlproduction
main.py  pyproject.toml  README.md  runtime  scripts  sqls  src  uv.lock


In [2]:
!chmod +x scripts/*.sh
!scripts/cloud_setup.sh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 55.6 MB/s eta 0:00:00:00:0100:01
Resolved 69 packages in 748ms                                        
# This file was autogenerated by uv via the following command:
#    uv pip compile pyproject.toml -o requirements.txt --extra-index-url https://pypi.org/simple --emit-index-url
--index-url https://pypi.org/simple
--extra-index-url https://pypi.org/simple

aiohappyeyeballs==2.6.1
    # via aiohttp
aiohttp==3.13.2
    # via fsspec
aiosignal==1.4.0
    # via aiohttp
annotated-types==0.7.0
    # via pydantic
attrs==25.4.0
    # via aiohttp
certifi==2025.11.12
    # via requests
charset-normalizer==3.4.4
    # via requests
filelock==3.20.1
    # via
    #   huggingface-hub
    #   torch
    #   transformers
frozenlist==1.8.0
    # via
    #   aiohttp
    #   aiosignal
fsspec==2025.12.0
    # via
    #   huggingface-hub
    #   lightning
    #   pytorch-lightning
    #   torch
hf-xet==1.2.0
    # via huggingface-hub
huggingface-hub==0

# Run

In [4]:
from runtime.ultils import gdown_folder
gdown_folder()

Retrieving folder contents


Retrieving folder 1aoddGUNcsdJuD6KtSRigVnqLW4uqeHin .ipynb_checkpoints
Processing file 1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE hallu_classes.csv
Processing file 1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT hallu_data.csv


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1pQevQ6Ors5DVuAXLxXiWkGh9LGi63DjE
To: /kaggle/working/mlproduction/data/hallu_classes.csv
100%|██████████| 25.0/25.0 [00:00<00:00, 76.1kB/s]
Downloading...
From: https://drive.google.com/uc?id=1BWfYcKTw1lBcEpeINvcsL2sW36hsadYT
To: /kaggle/working/mlproduction/data/hallu_data.csv
100%|██████████| 10.7M/10.7M [00:00<00:00, 58.8MB/s]
Download completed


Folder downloaded successfully.


# What is the dog doing?


In [6]:
import dagshub
dagshub.init(repo_owner='ollamagena', repo_name='mlpro', mlflow=True)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=7951183b-a413-4a9b-a6d8-5367d202d3ed&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=43ddc13091b1e823b162f52329921e1fe23f20bd862b48cbe28b8d733e6624f8




Accessing as ollamagena

Initialized MLflow to track repo "ollamagena/mlpro"

Repository ollamagena/mlpro initialized!

# Run

In [11]:
import pandas as pd
from src.dataloaders.ultils import train_test_split_df, load_label_enocder

df = pd.read_csv("data/hallu_data.csv")
df.head()
le = load_label_enocder("data/hallu_classes.csv")
X_train, X_test, y_train, y_test = train_test_split_df(df.iloc[:16], "label", le, 0.3, verbose=True)

X_train shape: (11, 4)
X_test shape: (5, 4)
y_train shape: (11,)
y_test shape: (5,)

Sample rows from X_train:
                                      id  \
2   10fca062-d343-4eca-8434-93c7a8aa5e0e   
13  13aa760a-1f6a-48a6-b82b-9106509409fe   

                                              context  \
2   Vị trí địa lý nằm giữa phương Đông và Địa Trun...   
13  Những nhà nghiên cứu kết luận rằng bệnh giang ...   

                                               prompt  \
2   Dựa trên việc Đế quốc Ottoman không bao giờ ki...   
13  Tại Bengal, nơi xuất hiện đầu tiên của đại dịc...   

                                             response  
2   Nền ẩm thực của Thổ Nhĩ Kỳ được xây dựng từ gi...  
13  Bệnh giang mai không bùng phát đầu tiên tại Be...  

Sample values from y_train: [0 2]


In [12]:
len(le.classes_)

3

In [17]:
import mlflow

mlflow.set_experiment("embedding-sklearn2")


2025/12/18 10:03:09 INFO mlflow.tracking.fluent: Experiment with name 'embedding-sklearn2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/9fc02da896174a8d950e6b059532f67a', creation_time=1766052189411, experiment_id='1', last_update_time=1766052189411, lifecycle_stage='active', name='embedding-sklearn2', tags={}>

In [18]:
from src.models.preprop_svm import Preprop, prep_remove_common_words_with_counting, agg_differential
from sklearn.svm import SVC
from mlflow.models import infer_signature
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

with mlflow.start_run(run_name="test run with SVC"):
    preprop_params = {
        "prepropfn": prep_remove_common_words_with_counting,
        "aggfn": agg_differential,
        "encoder_name": "bkai-foundation-models/vietnamese-bi-encoder"
    }

    clf_params = {
        "C": 1.0,
        "kernel": "rbf",
        "probability": True,
        "random_state": 42,
        "decision_function_shape": "ovr",   # one-vs-rest for multi-class
        "break_ties": True
    }

    mlflow.log_params(preprop_params|clf_params)

    preprop = Preprop(**preprop_params)
    x_train_emd = preprop.call_df(X_train,  8)
    x_test_emd = preprop.call_df(X_test, 8)

    svc = SVC(**clf_params)
    svc.fit(x_train_emd, y_train)
    y_pred = svc.predict(X_test)

    # Calculate and log metrics
    metrics = {
        "accuracy": accuracy_score(y_test, y_pred),
        "precision": precision_score(y_test, y_pred, average="weighted"),
        "recall": recall_score(y_test, y_pred, average="weighted"),
        "f1_score": f1_score(y_test, y_pred, average="weighted"),
    }
    mlflow.log_metrics(metrics)    

    signature = infer_signature(x_test_emd, svc.predict(x_test_emd))
    mlflow.sklearn.log_model(
        sk_model = svc,
        name="svc",
        signature=signature,
        input_example=x_train_emd[:5]
    )


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


🏃 View run test run with SVC at: https://dagshub.com/ollamagena/mlpro.mlflow/#/experiments/1/runs/5559c682aaed49319c35e214b91ec571
🧪 View experiment at: https://dagshub.com/ollamagena/mlpro.mlflow/#/experiments/1


TypeError: prep_remove_common_words_with_counting() missing 1 required positional argument: 'text2'